In [3]:
from bs4 import BeautifulSoup
import requests
import re
from collections import defaultdict
from pandas import read_table
import numpy as np
import math
from konlpy.tag import Okt
from konlpy import jvm, utils
okt=Okt()

#######################################
print("제품명을 입력하세요.\n")
x = input()

base_url = 'https://search.daum.net/search?w=blog&DA=PGD&enc=utf8&q={}&m=board&page={}'

s = []
url = []  # 나중에 필터링거친 주소들을 넣을 리스트

blog_link = []  # blog 링크

for i in range(1, 5):
    url = base_url.format(x, i)
    r = requests.get(url)
    c = r.content

    bsObject = BeautifulSoup(c, "html.parser")

    body = bsObject.select("#blogColl > div.coll_cont > ul > li > div > div > a")  # script body
    for link in body:
        blog_link.append(link.get("href"))


제품명을 입력하세요.

초록이


In [6]:
blog_data = []
for i in range(0, len(blog_link)):

    url = blog_link[i]
    r = requests.get(url)
    c = r.content
    s = BeautifulSoup(c, "html.parser")
    s = str(s)
    s = re.sub("<.+?>", "", s, 0, re.I | re.S)
    s = s.replace("\n", "")
    s = s.replace("(", "")
    s = s.replace(")", "")
    s = s.replace(":", "")
    s = s.replace(",", "")
    s = s.replace('"', "")
    s = s.split('.')
    result = []
    for k in range(0, len(s)):
        hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
        result.append(hangul.sub('', s[k]))

    sentence = ""

    for j in range(0, len(result)):
        sentence += result[j]

    blog_data.append(sentence)

In [9]:
compare=[]
for i in range(0,len(blog_data)):
    sentence_compare=[]
    p= okt.pos(blog_data[i])
    for a in range(0,len(p)):
        if p[a][1]!='Josa' and p[a][1]!='KoreanParticle':
            sentence_compare.append(p[a][0])
    compare.append(sentence_compare)

In [10]:
result_compare = []
for i in range(0, len(compare)):  
    sentence = ""
    for j in range(0, len(compare[i])):
        sentence =  sentence +" " +compare[i][j]
    result_compare.append(sentence)


In [ ]:
# NaiveBayesClassifier 참조
# https://repo.yona.io/yona-projects/yona/post/5
# https://gist.github.com/ratsgo/45d6eb4822ae27b01329e3b8c15c8f98

from collections import defaultdict
from pandas import read_table
import numpy as np
import math

class NaiveBayesClassifier:

    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []

    def load_corpus(self, path):
        corpus = read_table(path, sep=',', encoding='utf-8')
        corpus = np.array(corpus)
        return corpus

    def count_words(self, training_set):
        # 학습데이터는 영화리뷰 본문(doc), 평점(point)으로 구성
        counts = defaultdict(lambda : [0, 0])
        for doc, point in training_set:
            # 영화리뷰가 text일 때만 카운트
            if self.isNumber(doc) is False:
                # 리뷰를 띄어쓰기 단위로 토크나이징
                words = doc.split()
                for word in words:
                    counts[word][0 if point > 3.5 else 1] += 1
        return counts

    def isNumber(self, s):
        try:
            float(s)
            return True
        except ValueError:
            return False

    def word_probabilities(self, counts, total_class0, total_class1, k):
        # 단어의 빈도수를 [단어, p(w|긍정), p(w|부정)] 형태로 반환
        return [(w,
                 (class0 + k) / (total_class0 + 2*k),
                 (class1 + k) / (total_class1 + 2*k))
                for w, (class0, class1) in counts.items()]

    def class0_probability(self, word_probs, doc):
        # 별도 토크나이즈 안하고 띄어쓰기로만
        docwords = doc.split()

        # 초기값은 모두 0으로 처리
        log_prob_if_class0 = log_prob_if_class1 = 0.0

        # 모든 단어에 대해 반복
        for word, prob_if_class0, prob_if_class1 in word_probs:
            # 만약 리뷰에 word가 나타나면
            # 해당 단어가 나올 log 확률을 더해 줌
            if word in docwords:
                log_prob_if_class0 += math.log(prob_if_class0)
                log_prob_if_class1 += math.log(prob_if_class1)

            # 만약 리뷰에 word가 나타나지 않는다면
            # 해당 단어가 나오지 않을 log 확률을 더해 줌
            # 나오지 않을 확률은 log(1-나올 확률)로 계산
            else:
                log_prob_if_class0 += math.log(1.0 - prob_if_class0)
                log_prob_if_class1 += math.log(1.0 - prob_if_class1)

        prob_if_class0 = math.exp(log_prob_if_class0)
        prob_if_class1 = math.exp(log_prob_if_class1)
        return prob_if_class0 / (prob_if_class0 + prob_if_class1)

    def train(self, trainfile_path):
        training_set = self.load_corpus(trainfile_path)

        # 범주0(긍정)과 범주1(부정) 문서 수를 세어 줌
        num_class0 = len([1 for _, point in training_set if point > 3.5])
        num_class1 = len(training_set) - num_class0

        # train
        word_counts = self.count_words(training_set)
        self.word_probs = self.word_probabilities(word_counts,
                                                  num_class0,
                                                  num_class1,
                                                  self.k)

    def classify(self, doc):
        return self.class0_probability(self.word_probs, doc)

In [40]:
blog_date=[]
blog_date=result_compare
    
model = NaiveBayesClassifier()

model.train(trainfile_path=r'D:\add.csv')
a = []
for i in range(0, len(blog_data)):
    a.append(model.classify(blog_data[i]))

# print(a)

final_blog_link = []
for i in range(0, len(blog_data)):
    if a[i] < 0.2:
        final_blog_link.append(blog_link[i])

print("\n")
print("총 블로그 수 : ", len(blog_link))
print("실제 구매후기 블로그 수 : ", len(final_blog_link))
print("\n")

print("<블로그 URL>")
for i in range(0, len(final_blog_link)):
    print(final_blog_link[i])
#
#
#




총 블로그 수 :  40
실제 구매후기 블로그 수 :  10


<블로그 URL>
http://blog.naver.com/PostView.nhn?blogId=taemoms&logNo=221671576166
http://blog.naver.com/PostView.nhn?blogId=whitesoon1&logNo=221673072772
http://blog.naver.com/PostView.nhn?blogId=heimish94&logNo=221696548987
http://blog.naver.com/PostView.nhn?blogId=yeomi_snap&logNo=221695329978
http://blog.naver.com/PostView.nhn?blogId=nieah914&logNo=221686600038
http://blog.naver.com/PostView.nhn?blogId=pdyi&logNo=221697635352
http://blog.naver.com/PostView.nhn?blogId=leslie818&logNo=221677929753
http://blog.naver.com/PostView.nhn?blogId=lua910&logNo=221666799888
http://blog.naver.com/PostView.nhn?blogId=verymore9986&logNo=221701579851
http://blog.naver.com/PostView.nhn?blogId=stream_chung&logNo=221693102898


C:\Users\oink0\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead.
  # This is added back by InteractiveShellApp.init_path()
